# Parsing 10-K Report

In [13]:
import pathlib
ticker: str = "KO"
path_10k = pathlib.Path(f"/Users/rohan/Code/fintech_innovation_summer_task/sec-edgar-filings/AAPL/10-K/0000320193-95-000016/full-submission.txt")

In [41]:
import pathlib
ticker: str = "AAPL"
path_10k = pathlib.Path(f"../sec-edgar-filings/{ticker}/10-K/0000320193-95-000016/full-submission.txt")

In [64]:
import pathlib
ticker: str = "INTC"
path_10k = pathlib.Path(f"../sec-edgar-filings/INTC/10-K/0000050863-19-000007/full-submission.txt")

In [43]:
import pathlib
ticker: str = "DIS"
path_10k = pathlib.Path(f"../sec-edgar-filings/DIS/10-K/0001744489-20-000197/full-submission.txt")

# Parsing

In [65]:
from bs4 import BeautifulSoup
soup: BeautifulSoup = None
with open(path_10k, "r") as f:
    raw_text = f.read()
    soup = BeautifulSoup(raw_text)

In [ ]:
content = soup.findAll("document")

for doc in content:
    type_soup = doc.find("type")
    type_children = type_soup.contents
    type_str = type_children[0].strip()
    if type_str == "10-K":
        string_10k = type_children[1]

In [ ]:
ITEMS_10K = ["ITEM 1", "ITEM 1A", "ITEM 1B", "ITEM 2", "ITEM 3", "ITEM 4", "ITEM 5", "ITEM 6",
        "ITEM 7", "ITEM 7A", "ITEM 8", "ITEM 9", "ITEM 9A", "ITEM 9B", "ITEM 10",
        "ITEM 11", "ITEM 12", "ITEM 13", "ITEM 14", "ITEM 15"]

In [ ]:
from dataclasses import dataclass
@dataclass
class Entry:
    company: str
    year: int
    item: str
    text: str

In [59]:
import re


def handle_html_format(string_10k_text: str):
    item_to_index = {}
    last_added = None
    for i, item in enumerate(ITEMS_10K):
        first, second = item.split(" ")
        
        text_regex = f">{first}(\\s|&nbsp;|\\n)*{second}."
        opts = list(re.finditer(f"{text_regex}\\.", string_10k_text))
        if len(opts) < 1:
            first = first[0] + first[1:].lower()
            text_regex = f">{first}(\\s|&nbsp;|\\n)*{second}."
            opts = list(re.finditer(f"{text_regex}\\.", string_10k_text))

        if len(opts) > 0:
            m = opts[-1]
            item_to_index[item] = (m.end(), len(string_10k_text))
            if i > 0:
                start, _ = item_to_index[last_added]
                item_to_index[last_added] = start, m.start()
            last_added = item

    return item_to_index

def handle_nonhtml_format(string_10k_text: str):
    item_to_index = {}
    last_added = None

    for i, item in enumerate(ITEMS_10K):
        first, second = item.split(" ")
        
        text_regex = f"{first}(\\s|&nbsp;|\\n)*{second}\\."
        opts = list(re.finditer(f"{text_regex}", string_10k_text))
        if len(opts) < 1:
            first_lowercase = first[0] + first[1:].lower()
            text_regex = f"{first_lowercase}(\\s|&nbsp;|\\n)*{second}\\."
            opts = list(re.finditer(f"{text_regex}", string_10k_text))
        if len(opts) > 0:
            m = opts[0]
            item_to_index[item] = (m.end(), len(string_10k_text))
            if i > 0:
                start, _ = item_to_index[last_added]
                item_to_index[last_added] = start, m.start()
            last_added = item

    return item_to_index
tags_in_doc = len(string_10k.findAll())
if tags_in_doc > 500:
    item_to_index = handle_html_format(string_10k.text)
else:
    item_to_index = handle_nonhtml_format(string_10k.text)

entries = []
for item in item_to_index:
    start, end = item_to_index[item]
    text = string_10k.text[start:end].strip()
    
    entries.append(
        Entry(
            None,
            None,
            item,
            text
        )
    )

for e in entries:
    if len(e.text) < 1:
        print(f"Missing Text for {e.company} - {e.year} - {e.item}")




In [60]:
item_to_index

{'ITEM 1': (3901, 29380),
 'ITEM 2': (29387, 31475),
 'ITEM 3': (31482, 31752),
 'ITEM 4': (31759, 38635),
 'ITEM 5': (38645, 39408),
 'ITEM 6': (39415, 41566),
 'ITEM 7': (41573, 87579),
 'ITEM 8': (87586, 167859),
 'ITEM 9': (167867, 168030),
 'ITEM 10': (168038, 168567),
 'ITEM 11': (168575, 169154),
 'ITEM 12': (169162, 169552),
 'ITEM 13': (169560, 171408),
 'ITEM 14': (171416, 193903)}

In [58]:
string_10k.text.index("Item 10.") + 8

168038

In [56]:
string_10k.text.index("Item 11.")

168567

In [29]:
entries = []
for item in item_to_index:
    start, end = item_to_index[item]
    text = string_10k.text[start:end].strip()
    
    entries.append(
        Entry(
            "company_name",
            "year",
            item,
            text
        )
    )

In [30]:
item_to_index

{}

In [11]:
for item_name, item_indices in item_to_index.items():
    start, end = item_indices
    print(item_name)
    print(string_10k.text[start:end])
    print("**"* 10)

ITEM 1
 BUSINESS     
        The Coca-Cola Company is the largest manufacturer, distributor and marketer of nonalcoholic beverage concentrates and syrups in the world. Finished
beverage products bearing our trademarks, sold in the United States since 1886, are now sold in more than 200 countries and include the leading soft drink products in most of these countries. In this
report, the terms "Company," "we," "us" or "our" mean The Coca-Cola Company and all subsidiaries included in our consolidated financial statements. 
        Our
business is nonalcoholic beveragesprincipally carbonated soft drinks, but also a variety of noncarbonated beverages. We manufacture beverage concentrates and syrups, as
well as some finished beverages, which we sell to bottling and canning operations, distributors, fountain wholesalers and some fountain retailers. We also produce, market and distribute juices and
juice drinks and certain water products. In addition, we have ownership interests in numerous 